# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Oct 27 2022  8:58AM  249491        10  0086119118  ISDIN Corporation   
1  Oct 27 2022  8:58AM  249491        10  0086119182  ISDIN Corporation   
2  Oct 27 2022  8:58AM  249491        10  0086119208  ISDIN Corporation   
3  Oct 27 2022  8:58AM  249491        10  0086119213  ISDIN Corporation   
4  Oct 27 2022  8:58AM  249491        10  0086119214  ISDIN Corporation   
5  Oct 27 2022  8:58AM  249491        10  0086119219  ISDIN Corporation   
6  Oct 27 2022  8:58AM  249491        10  0086119225  ISDIN Corporation   
7  Oct 27 2022  8:58AM  249491        10  0086119227  ISDIN Corporation   
8  Oct 27 2022  8:58AM  249491        10  0086119232  ISDIN Corporation   
9  Oct 27 2022  8:58AM  249491        10  0086119243  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
17  249485       Released          1
18  249487       Released          1
19  249488       Released         14
20  249490       Released         47
21  249491       Released         11

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
249485                NaN       1.0
249487                NaN       1.0
249488                NaN      14.0
249490                NaN      47.0
249491                NaN      11.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
248349                0.0       1.0
248599                0.0       1.0
249407                0.0       1.0
249418                0.0       1.0
249447                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
248349                  0         1
248599                  0         1
249407                  0         1
249418                  0         1
249447                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               248349          0         1
1               248599          0         1
2               249407          0         1
3               249418          0         1
4               249447          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing  Released
0               248349                   1
1               248599                   1
2               249407                   1
3               249418                   1
4               249447                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Oct 27 2022  8:58AM  249491        10               ISDIN Corporation
11  Oct 27 2022  8:58AM  249490        10               ISDIN Corporation
58  Oct 27 2022  8:43AM  249488        10               ISDIN Corporation
72  Oct 27 2022  8:35AM  249487        19                  Innogenix, LLC
73  Oct 27 2022  8:34AM  249485        20     Reliable 1 Laboratories LLC
74  Oct 27 2022  8:32AM  249484        12                       Hush Hush
94  Oct 27 2022  8:32AM  249482        12                       Hush Hush
95  Oct 27 2022  8:21AM  249479        16  Sartorius Bioprocess Solutions
96  Oct 27 2022  8:20AM  249478        16  Sartorius Bioprocess Solutions
97  Oct 27 2022  7:43AM  249473        19             AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Oct 27 2022  8:58AM  249491        10               ISDIN Corporation   
1  Oct 27 2022  8:58AM  249490        10               ISDIN Corporation   
2  Oct 27 2022  8:43AM  249488        10               ISDIN Corporation   
3  Oct 27 2022  8:35AM  249487        19                  Innogenix, LLC   
4  Oct 27 2022  8:34AM  249485        20     Reliable 1 Laboratories LLC   
5  Oct 27 2022  8:32AM  249484        12                       Hush Hush   
6  Oct 27 2022  8:32AM  249482        12                       Hush Hush   
7  Oct 27 2022  8:21AM  249479        16  Sartorius Bioprocess Solutions   
8  Oct 27 2022  8:20AM  249478        16  Sartorius Bioprocess Solutions   
9  Oct 27 2022  7:43AM  249473        19             AdvaGen Pharma, Ltd   

  Executing  Released  
0                  11  
1                  47  
2                  14  
3                   1  
4                   1  
5                  20  
6                   1  
7                   1  
8                   1  
9                  25

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Oct 27 2022  8:58AM  249491        10               ISDIN Corporation   
1  Oct 27 2022  8:58AM  249490        10               ISDIN Corporation   
2  Oct 27 2022  8:43AM  249488        10               ISDIN Corporation   
3  Oct 27 2022  8:35AM  249487        19                  Innogenix, LLC   
4  Oct 27 2022  8:34AM  249485        20     Reliable 1 Laboratories LLC   
5  Oct 27 2022  8:32AM  249484        12                       Hush Hush   
6  Oct 27 2022  8:32AM  249482        12                       Hush Hush   
7  Oct 27 2022  8:21AM  249479        16  Sartorius Bioprocess Solutions   
8  Oct 27 2022  8:20AM  249478        16  Sartorius Bioprocess Solutions   
9  Oct 27 2022  7:43AM  249473        19             AdvaGen Pharma, Ltd   

   Released Executing  
0        11            
1        47            
2        14            
3         1            
4         1            
5        20            
6         1            
7         1            
8         1            
9        25

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Oct 27 2022  8:58AM  249491        10            ISDIN Corporation   
1  Oct 27 2022  8:58AM  249490        10            ISDIN Corporation   
2  Oct 27 2022  8:43AM  249488        10            ISDIN Corporation   
3  Oct 27 2022  8:35AM  249487        19               Innogenix, LLC   
4  Oct 27 2022  8:34AM  249485        20  Reliable 1 Laboratories LLC   

   Released Executing  
0        11            
1        47            
2        14            
3         1            
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                     Customer  \
0  Oct 27 2022  8:58AM  249491        10            ISDIN Corporation   
1  Oct 27 2022  8:58AM  249490        10            ISDIN Corporation   
2  Oct 27 2022  8:43AM  249488        10            ISDIN Corporation   
3  Oct 27 2022  8:35AM  249487        19               Innogenix, LLC   
4  Oct 27 2022  8:34AM  249485        20  Reliable 1 Laboratories LLC   

   Released  Executing  
0        11        NaN  
1        47        NaN  
2        14        NaN  
3         1        NaN  
4         1        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Oct 27 2022  8:58AM  249491        10            ISDIN Corporation   
1  Oct 27 2022  8:58AM  249490        10            ISDIN Corporation   
2  Oct 27 2022  8:43AM  249488        10            ISDIN Corporation   
3  Oct 27 2022  8:35AM  249487        19               Innogenix, LLC   
4  Oct 27 2022  8:34AM  249485        20  Reliable 1 Laboratories LLC   

   Released  Executing  
0        11        0.0  
1        47        0.0  
2        14        0.0  
3         1        0.0  
4         1        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                             
10         997931        77        0.0
12         997843        23        0.0
16         997893         4        0.0
19         997876        30        9.0
20         249485         1        0.0
21         995806         4        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse      id  Released  Executing
0        10  997931        77        0.0
1        12  997843        23        0.0
2        16  997893         4        0.0
3        19  997876        30        9.0
4        20  249485         1        0.0
5        21  995806         4        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10        77        0.0
1        12        23        0.0
2        16         4        0.0
3        19        30        9.0
4        20         1        0.0
5        21         4        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   77.0
1        12  Released   23.0
2        16  Released    4.0
3        19  Released   30.0
4        20  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12   16    19   20   21
Status                                    
Executing   0.0   0.0  0.0   9.0  0.0  0.0
Released   77.0  23.0  4.0  30.0  1.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12   16    19   20   21
0          Executing   0.0   0.0  0.0   9.0  0.0  0.0
1           Released  77.0  23.0  4.0  30.0  1.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12   16    19   20   21
0  Executing   0.0   0.0  0.0   9.0  0.0  0.0
1   Released  77.0  23.0  4.0  30.0  1.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()